In [2]:
!pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/8f/87/5969092159207f583481ad80a03f09e2d4af1ebd197f4530ca4e906c947e/scikit_learn-1.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/ef/1b/7538792254aec6850657d5b940fd05fe60582af829ffe40d6c054f065f34/scipy-1.11.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01
  Obtaining dependency information for joblib>=1.1.1 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-an

In [11]:
import numpy as np

from sklearn.preprocessing import StandardScaler
import sklearn.svm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import cross_validate
import pickle
import pandas as pd

In [4]:
verbose = True
random_seed = 1234
cv_param_folds = 5
cv_select_folds = 5

n_jobs = 1

In [7]:
pipeline = []

pipeline.append(('scaler',StandardScaler()))
pipeline.append(('svc',sklearn.svm.SVC(verbose=verbose)))

model_pipeline = Pipeline(pipeline)

In [8]:
param_grid = [{'C': 10.**np.arange(-3,3,2),
               'gamma': 10.**np.arange(-3,1,1)}]

def pipeline_param_grid(param_grid,model_name):
    pg = []
    for x in param_grid:
        d = {}
        for k,v in x.items():
            d[model_name+'__'+k] = v
        pg += [d]

    return pg

param_grid = pipeline_param_grid(param_grid,'svc')

mcc = sklearn.metrics.make_scorer(sklearn.metrics.matthews_corrcoef)

inner_cv = StratifiedKFold(n_splits=cv_param_folds,
                           shuffle=True,
                           random_state=random_seed)

clf = GridSearchCV(estimator = model_pipeline,
                param_grid = param_grid,
                cv = inner_cv,
                verbose = verbose,
                scoring = dict(accuracy='accuracy',
                                f1_macro='f1_macro',
                                mcc=mcc),
                refit = 'mcc')

In [12]:
with open('hurst_data.pickle','rb') as fp:
    data = pickle.load(fp)

In [16]:
df = pd.read_csv('data_labels.csv')

In [29]:
df['CDR'].unique()

array([0. , 0.5, nan, 1. , 2. ])

In [32]:
df['CDR'].dropna()

0      0.0
1      0.0
2      0.5
8      0.0
9      0.0
      ... 
411    0.5
412    0.5
413    0.0
414    0.0
415    0.0
Name: CDR, Length: 235, dtype: float64

In [25]:
df[df.loc[:,"ID"].isin(list(data.keys()))]

,ID,M/F,Hand,Age,Educ,SES,MMSE,CDR,eTIV,nWBV,ASF,Delay
43,OAS1_0047_MR1,F,R,57,NaN,NaN,NaN,NaN,1408,0.784,1.247,NaN
90,OAS1_0097_MR1,M,R,23,NaN,NaN,NaN,NaN,1568,0.816,1.119,NaN
120,OAS1_0130_MR1,M,R,68,3.0,3.0,26.0,0.0,1444,0.789,1.216,NaN
123,OAS1_0133_MR1,F,R,65,5.0,2.0,30.0,0.0,1277,0.814,1.374,NaN
195,OAS1_0213_MR1,F,R,48,NaN,NaN,NaN,NaN,1332,0.801,1.318,NaN
198,OAS1_0217_MR1,F,R,78,4.0,3.0,27.0,0.5,1393,0.692,1.260,NaN
244,OAS1_0271_MR1,F,R,89,2.0,4.0,27.0,0.0,1329,0.740,1.320,NaN
259,OAS1_0287_MR1,F,R,78,3.0,3.0,21.0,0.5,1194,0.694,1.470,NaN
326,OAS1_0361_MR1,M,R,20,NaN,NaN,NaN,NaN,1485,0.842,1.182,NaN
370,OAS1_0408_MR1,F,R,22,NaN,NaN,NaN,NaN,1518,0.861,1.156,NaN


In [15]:
list(data.keys())

['OAS1_0047_MR1',
 'OAS1_0097_MR1',
 'OAS1_0217_MR1',
 'OAS1_0271_MR1',
 'OAS1_0423_MR1',
 'OAS1_0361_MR1',
 'OAS1_0287_MR1',
 'OAS1_0130_MR1',
 'OAS1_0133_MR1',
 'OAS1_0213_MR1',
 'OAS1_0408_MR1']

In [ ]:
X = 

In [ ]:
outer_cv = StratifiedKFold(n_splits=cv_select_folds,
                           shuffle=True,
                           random_state=random_seed)

cv_select_results = cross_validate(estimator = clf,
                                       X = X,
                                       y = y,
                                       cv = outer_cv,
                                       n_jobs = n_jobs,
                                       verbose = verbose,
                                       return_estimator = True,
                                       return_train_score = True,
                                       scoring = dict(accuracy='accuracy',
                                                      f1_macro='f1_macro',
                                                      mcc=mcc))


In [ ]:
cv_select_results

In [ ]:
best_model = clf.fit(X,y)